# 27 · Descriptores de forma (contornos)

Imports y ruta de imagen.

In [ ]:
import cv2, numpy as np, matplotlib.pyplot as plt, pandas as pd

# IMG_PATH = r"C:\\Users\\20808\\Documents\\Repositorios\\Vision_en_Robotica\\imagenes\\pildoras.png"
# IMG_PATH = r"C:\\Users\\20808\\Documents\\Repositorios\\Vision_en_Robotica\\imagenes\\pildoras1.png"
IMG_PATH = r"C:\\Users\\20808\\Documents\\Repositorios\\Vision_en_Robotica\\imagenes\\pildoras2.png"
# IMG_PATH = r"C:\\Users\\20808\\Documents\\Repositorios\\Vision_en_Robotica\\imagenes\\pildoras3.png"

Carga y visualización.

In [ ]:
bgr = cv2.imread(IMG_PATH, cv2.IMREAD_COLOR)
if bgr is None:
    raise FileNotFoundError(f"No se pudo cargar: {IMG_PATH}")
rgb = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)
plt.figure(figsize=(6,6)); plt.imshow(rgb); plt.title("RGB"); plt.axis("off"); plt.show()

Conversión a gris, suavizado y umbralización (Otsu) con polaridad automática.

In [ ]:
gray = cv2.cvtColor(bgr, cv2.COLOR_BGR2GRAY)
gray_blur = cv2.GaussianBlur(gray, (5,5), 0)
use_inverse = gray_blur.mean() > 127
mode = cv2.THRESH_BINARY_INV if use_inverse else cv2.THRESH_BINARY
_, bin_img = cv2.threshold(gray_blur, 0, 255, mode + cv2.THRESH_OTSU)
plt.figure(figsize=(6,6)); plt.imshow(bin_img, cmap="gray"); plt.title(f"Binaria (Otsu) modo={'INV' if use_inverse else 'BIN'}"); plt.axis("off"); plt.show()

Detección de contornos externos y visualización.

In [ ]:
contours, hierarchy = cv2.findContours(bin_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
vis = rgb.copy()
cv2.drawContours(vis, contours, -1, (255,0,0), 2)
plt.figure(figsize=(6,6)); plt.imshow(vis); plt.title(f"Contornos externos: {len(contours)}"); plt.axis("off"); plt.show()

Funciones auxiliares para descriptores.

In [ ]:
def contour_descriptors(cnt):
    area = cv2.contourArea(cnt)
    perim = cv2.arcLength(cnt, True)
    x,y,w,h = cv2.boundingRect(cnt)
    rect_area = w*h
    # Circularidad: 4πA / P^2 (1 = círculo)
    circularity = (4*np.pi*area)/(perim**2) if perim>0 else 0
    # Solidez: A / A_casco (1 = convexo)
    hull = cv2.convexHull(cnt)
    hull_area = cv2.contourArea(hull)
    solidity = (area/hull_area) if hull_area>0 else 0
    # Relación de aspecto: w/h
    aspect = (w/h) if h>0 else 0
    # Excentricidad aproximada a partir de elipse ajustada
    ecc = np.nan
    if len(cnt) >= 5:
        (cx,cy),(MA,ma),angle = cv2.fitEllipse(cnt)
        a = max(MA,ma)/2.0; b = min(MA,ma)/2.0
        if a>0: ecc = np.sqrt(1 - (b*b)/(a*a))
    # Momentos de Hu (log-scale con signo preservado)
    hu = cv2.HuMoments(cv2.moments(cnt)).flatten()
    hu_log = np.array([-np.sign(h)*np.log10(abs(h)) if h!=0 else 0 for h in hu])
    return dict(area=area, perimetro=perim, x=x, y=y, w=w, h=h, circularidad=circularity,
                solidez=solidity, aspecto=aspect, excentricidad=ecc,
                hu1=hu_log[0], hu2=hu_log[1], hu3=hu_log[2], hu4=hu_log[3], hu5=hu_log[4], hu6=hu_log[5], hu7=hu_log[6])

Cálculo de descriptores por contorno y tabla de resultados.

In [ ]:
rows = []
for i, cnt in enumerate(contours, start=1):
    d = contour_descriptors(cnt)
    d['ID'] = i
    rows.append(d)
df = pd.DataFrame(rows).set_index('ID').sort_index()
pd.set_option('display.float_format', lambda x: f'{x:.4f}')
df

Visualización: color por circularidad; IDs y bounding boxes.

In [ ]:
vis_desc = rgb.copy()
min_c, max_c = float(df['circularidad'].min()), float(df['circularidad'].max())
def color_map(val, vmin, vmax):
    if vmax<=vmin: return (255,255,255)
    t = (val - vmin)/(vmax - vmin)
    # mapa simple azul->verde->rojo
    r = int(255*t)
    g = int(255*(1-abs(t-0.5)*2))
    b = int(255*(1-t))
    return (r,g,b)

for i, cnt in enumerate(contours, start=1):
    x,y,w,h = cv2.boundingRect(cnt)
    cval = float(df.loc[i,'circularidad']) if i in df.index else 0.0
    col = color_map(cval, min_c, max_c)
    cv2.drawContours(vis_desc, [cnt], -1, col, 2)
    cv2.rectangle(vis_desc, (x,y), (x+w,y+h), (0,255,0), 1)
    cx = int(x + w/2); cy = int(y + h/2)
    cv2.putText(vis_desc, str(i), (cx, cy), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,0), 2, cv2.LINE_AA)

plt.figure(figsize=(6,6)); plt.imshow(vis_desc); plt.title("Contornos coloreados por circularidad"); plt.axis("off"); plt.show()

- Circularidad ≈ 1 indica formas cercanas a círculo; valores menores indican formas alargadas.
- Solidez < 1 indica concavidades o contornos con irregularidades respecto a su casco convexo.
- Excentricidad en [0,1); cercana a 0 ≈ círculo, cercana a 1 ≈ elipse muy alargada.
- Momentos de Hu (log) son invariantes a traslación, escala y rotación (sensibles al ruido).
